# SBTi-Finance Tool - Quick Temperature Score Calculation
This notebook provides a simple example of the SBTi-Finance Tool. It shows how to use it to calculate the temperature score for companies, aggregate them to a portfolio level to get the portfolio temperature score. It also shows you how to calculate the portfolio coverage. 

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/) for more details. 

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 

### Install the SBTi Python module
This is only required if you have not already installed the module.

In [ ]:
%pip install sbti-finance-tool

In [ ]:
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd
from datetime import datetime
import requests

## Download the dummy data provider
We have prepared dummy data for you to be able to run the tool as it is to familiarise yourself with how it works. To use your own data; please check out to the [Data Requirements section](https://sciencebasedtargets.github.io/SBTi/DataRequirements.html) of the technical documentation for more details on data requirements and formatting. 

*The dummy data may include some company names, but the data associated with those company names is completely random and any similarities with real world data is purely coincidental. 


In [ ]:
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_portfolio.csv", "data/example_portfolio.csv")

##### Logging
The SBTi module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [ ]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

## Create a data provider
Data providers let you connect to the data source of your choice. In this case we are connecting to Excel as a data provider. For all available dataproviders check the implementation [here](https://github.com/ScienceBasedTargets/SBTi-finance-tool/tree/master/SBTi/data)

In [ ]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

## Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "company_id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

Please see the technical documentation on [Data Legends](https://sciencebasedtargets.github.io/SBTi/Legends.html#) for details on data requirements. 

In [ ]:
# Load the portfolio data
portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")

# First check what columns are actually in the dataframe
print("Original columns:", portfolio.columns.tolist())

# Change the column names to match the new SBTi CTA format

rename_dict = {}
if "Company Name" in portfolio.columns:
    rename_dict["Company Name"] = "company_name"
if "ISIN" in portfolio.columns:
    rename_dict["ISIN"] = "isin" 
if "LEI" in portfolio.columns:
    rename_dict["LEI"] = "lei"
if "Sector" in portfolio.columns:
    rename_dict["Sector"] = "sector"
if "Target" in portfolio.columns:
    rename_dict["Target"] = "full_target_language"
if "Net-Zero Committed" in portfolio.columns:
    rename_dict["Net-Zero Committed"] = "net_zero_status"
if "Near term - Target Status" in portfolio.columns:
    rename_dict["Near term - Target Status"] = "near_term_status"
if "Target Classification" in portfolio.columns:
    rename_dict["Target Classification"] = "target_classification_long"
if "Extension" in portfolio.columns:
    rename_dict["Extension"] = "reason_for_extension_or_removal"
if "Date" in portfolio.columns:
    rename_dict["Date"] = "date_updated"

# Apply the renaming
portfolio = portfolio.rename(columns=rename_dict)

# Handle legacy column names if they exist
if 'company_isin' in portfolio.columns:
    portfolio.rename(columns={'company_isin': 'isin'}, inplace=True)
if 'company_lei' in portfolio.columns:
    portfolio.rename(columns={'company_lei': 'lei'}, inplace=True)

# Ensure required columns exist
required_columns = ['company_id', 'company_name', 'isin', 'lei', 'investment_value']
for col in required_columns:
    if col not in portfolio.columns:
        portfolio[col] = None

# Fix datatype of lei column - it might be numeric and need to be converted to string
if 'lei' in portfolio.columns:
    portfolio['lei'] = portfolio['lei'].astype(str)
if 'isin' in portfolio.columns:
    portfolio['isin'] = portfolio['isin'].astype(str)

# Check for duplicate values in the 'company_id' column
duplicate_ids = portfolio[portfolio.duplicated('company_id', keep=False)]
if not duplicate_ids.empty:
    print("Error: Duplicate values found in the 'company_id' column:")
    print(duplicate_ids)
else:
    print("No duplicate values found in the 'company_id' column.")

# Print final columns to verify
print("Final columns:", portfolio.columns.tolist())

In [ ]:
portfolio.head(5)

To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://sciencebasedtargets.github.io/SBTi-finance-tool/autoapi/SBTi/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [ ]:
companies = SBTi.utils.dataframe_to_portfolio(portfolio)

Import the CTA and set up a SBTi target frame of reference.

In [ ]:
#Provides an absolute frame of reference for SBTi targets so that they are considered as cardinal compared to others in the calculation of temperature scores.
def inject_sbti_validation_for_timeframe_scope_data(amended_portfolio, original_portfolio, debug=True):
    """
    Specially designed for the SBTi tool where amended_portfolio contains multiple rows 
    per company (one for each time frame and scope combination).
    """
    if 'sbti_validated' not in original_portfolio.columns:
        print("⚠ No 'sbti_validated' column found in original portfolio")
        return amended_portfolio
    
    # Store original values before modification
    original_validated_count = original_portfolio['sbti_validated'].sum()
    original_companies_count = len(original_portfolio)
    
    # Get count of unique companies in amended portfolio
    unique_companies_amended = amended_portfolio['company_id'].nunique()
    
    if debug:
        print(f"Original portfolio: {original_companies_count} companies, {original_validated_count} validated")
        print(f"Amended portfolio: {len(amended_portfolio)} rows, {unique_companies_amended} unique companies")
        
        # Check for duplicated rows by company_id
        if len(amended_portfolio) > unique_companies_amended:
            print(f"Multiple rows per company detected in amended portfolio")
            print(f"Rows per company: {len(amended_portfolio) / unique_companies_amended:.2f}")
            
            # Show distribution of time_frame and scope if they exist
            if 'time_frame' in amended_portfolio.columns:
                print("\nTime frame distribution:")
                print(amended_portfolio['time_frame'].value_counts())
            if 'scope' in amended_portfolio.columns:
                print("\nScope distribution:")
                print(amended_portfolio['scope'].value_counts())
    
    # Create a validation mapping
    validation_map = dict(zip(original_portfolio['company_id'], original_portfolio['sbti_validated']))
    
    # Apply validation to all rows in amended portfolio
    amended_portfolio['sbti_validated'] = amended_portfolio['company_id'].map(validation_map).fillna(False)
    
    # Count unique validated companies after modification
    validated_companies = amended_portfolio[amended_portfolio['sbti_validated']]['company_id'].nunique()
    
    # Print validation summary
    print(f"\nOriginal validated companies: {original_validated_count}")
    print(f"Unique companies validated in amended portfolio: {validated_companies}")
    
    if original_validated_count == validated_companies:
        print("✓ CTA validation successfully preserved at company level")
    else:
        print("⚠ CTA validation mismatch at company level")
        
        # Additional debugging
        if debug:
            print("\nChecking for specific discrepancies...")
            # Get list of company IDs that should be validated
            original_validated_ids = set(original_portfolio[original_portfolio['sbti_validated']]['company_id'])
            amended_validated_ids = set(amended_portfolio[amended_portfolio['sbti_validated']]['company_id'])
            
            missing_validations = original_validated_ids - amended_validated_ids
            extra_validations = amended_validated_ids - original_validated_ids
            
            if missing_validations:
                print(f"Companies that should be validated but aren't: {len(missing_validations)}")
                print(missing_validations)
            
            if extra_validations:
                print(f"Companies that shouldn't be validated but are: {len(extra_validations)}")
                print(extra_validations)
    
    return amended_portfolio

In [ ]:
# STANDALONE SBTi VALIDATION - Download and process CTA data
print("Downloading SBTi Companies Taking Action (CTA) data...")
CTA_FILE_URL = "https://cdn.sciencebasedtargets.org/download/target-dashboard"

try:
    resp = requests.get(CTA_FILE_URL)
    if resp.status_code == 200:
        cta_file = pd.read_excel(resp.content)
        print(f"Downloaded CTA data with {len(cta_file)} rows")
        
        # Extract relevant columns
        targets = cta_file[['company_name', 'isin', 'lei', 'action', 'target', 'date_published']]
        
        # Filter for companies with targets
        companies_with_targets = targets[targets['action'] == 'Target']
        
        # Get unique identifiers
        all_isin_set = set(companies_with_targets['isin'].dropna())
        all_lei_set = set(companies_with_targets['lei'].dropna())
        
        # Create a set of lowercase company names
        companies_with_targets['company_name_lower'] = companies_with_targets['company_name'].str.lower()
        company_name_set = set(companies_with_targets['company_name_lower'].dropna())
        
        # Add portfolio columns if they don't exist
        if 'isin' not in portfolio.columns and 'company_isin' in portfolio.columns:
            portfolio['isin'] = portfolio['company_isin']
        if 'lei' not in portfolio.columns and 'company_lei' in portfolio.columns:
            portfolio['lei'] = portfolio['company_lei']
        
        # Function to check if ISIN, LEI, or company name is validated
        def is_validated(row):
            # First check LEI
            if pd.notna(row.get('lei')) and str(row.get('lei')).lower() in [str(x).lower() for x in all_lei_set]:
                return True
            
            # Then check ISIN
            if pd.notna(row.get('isin')) and str(row.get('isin')).lower() in [str(x).lower() for x in all_isin_set]:
                return True
            
            # Finally check company name
            if pd.notna(row.get('company_name')):
                company_name_lower = str(row.get('company_name')).lower()
                if company_name_lower in company_name_set:
                    return True
            
            return False
        
        # Add the validated column to the portfolio
        portfolio['sbti_validated'] = portfolio.apply(is_validated, axis=1)
        
        # Convert portfolio to company objects again (after adding sbti_validated)
        companies = SBTi.utils.dataframe_to_portfolio(portfolio)
        
        # Print validation summary
        validated_count = portfolio['sbti_validated'].sum()
        print(f"Companies with SBTi-validated targets: {validated_count} out of {len(portfolio)} ({validated_count/len(portfolio)*100:.2f}%)")
        
        if 'investment_value' in portfolio.columns:
            total_investment = portfolio['investment_value'].sum()
            validated_investment = portfolio[portfolio['sbti_validated']]['investment_value'].sum()
            print(f"Portfolio coverage by investment value: {validated_investment/total_investment*100:.2f}%")
    else:
        print(f"Failed to download CTA file: HTTP {resp.status_code}")
except Exception as e:
    print(f"Error processing CTA file: {str(e)}")

# Update provider data with our validation results
print("Updating provider data with validated companies...")
try:
    # Get the fundamental_data from provider
    if hasattr(provider, 'data') and 'fundamental_data' in provider.data:
        # Create a mapping of company_id to validation status
        validation_map = portfolio[['company_id', 'sbti_validated']].set_index('company_id')['sbti_validated'].to_dict()
        
        # Update sbti_validated in provider data
        updated_count = 0
        for idx, row in provider.data['fundamental_data'].iterrows():
            company_id = row['company_id']
            if company_id in validation_map:
                # Ensure we're setting a proper boolean value
                is_validated = bool(validation_map[company_id])
                provider.data['fundamental_data'].at[idx, 'sbti_validated'] = is_validated
                updated_count += 1
        
        # Force the sbti_validated column to be boolean type
        provider.data['fundamental_data']['sbti_validated'] = provider.data['fundamental_data']['sbti_validated'].astype(bool)
        
        print(f"Updated sbti_validated for {updated_count} companies in provider data")
    else:
        print("Provider does not have expected data structure - sbti_validated flags may be overwritten")
except Exception as e:
    print(f"Error updating provider data: {str(e)}")

## Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [ ]:
temperature_score = TemperatureScore(                  # all available options:
    time_frames=list(SBTi.interfaces.ETimeFrames),     # ETimeFrames: SHORT MID and LONG
    scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3],    # EScopes: S3, S1S2 and S1S2S3
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)
# Preserve the CTA validation from our direct download
amended_portfolio = inject_sbti_validation_for_timeframe_scope_data(amended_portfolio, portfolio, debug=True)

For every company the tool assigns a score for all the requested timeframe and scope combinations. In this example we used the full set resulting in 9 scores per company as displayed below:

In [ ]:
amended_portfolio[['company_name', 'time_frame', 'scope', 'temperature_score']].head(9)

## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. Here we'll use the "Weighted Average Temperature Score" (WATS) by initializing the TemperatureScore Object with PortfolioAggregationMethod.WATS. For more details, please refer to notebook 4 (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/4_portfolio_aggregations.ipynb) or [GitHub](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/4_portfolio_aggregations.ipynb)) and the [methodology document](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf) sections 3.2. The temperature scores are calculated per time-frame/scope combination.


In [ ]:
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)

In [ ]:
# Here we cast a ScoreAggregation object to a Pandas Dataframe for viewing the temp scores in a human readable way
pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))

## Portfolio coverage

The portfolio coverage provides insights in the proportion of the portfolio that has set SBTi-approved GHG emissions reduction targets. Only companies with SBTi-status "Approved" are included in the portfolio coverage.

To calculate the portfolio coverage we use the same aggregation methods we use for the Portfolio Temperature Score. In this example we use the "Weighted Average Temperature Score" (WATS). For more details on aggregation methods and the portfolio coverage method, please refer to the [methodology document](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf) sections 3.2 and also turn to notebook 4 (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/4_portfolio_aggregations.ipynb) or [GitHub](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/4_portfolio_aggregations.ipynb)) for more aggregation examples.

In [ ]:
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Portfolio coverage is: {c:.2f}%".format(c=coverage))